# Assignment #2: Language models
Author: Pierre Nugues

## Objectives

The objectives of this assignment are to:
* Write a program to find n-gram statistics
* Compute the probability of a sentence
* Know what a language model is
* Write a short report of 1 to 2 pages on the assignment

## Submission

Once you have written all the missing code and run all the cells, you will submit your notebook to an automatic marking system. Do not erase the content of the cells as we will possibly check your programs manually.
The submission instructions are at the bottom of the notebook.

## Organization

* Each group will have to write Python programs to count unigrams, bigrams, and trigrams in a corpus of approximately one million words and to determine the probability of a sentence.
* You can test you regular expression using the regex101.com site
* Each student will have to write a short report of one to two pages and comment briefly the results. In your report, you must produce the tabulated results of your analysis as described below.

## Programming

### Imports

Some imports you may need. Add others as needed.

In [1]:
import bz2
import math
import os
import regex as re
import requests
import sys
from zipfile import ZipFile

### Collecting and analyzing a corpus

Retrieve a corpus of novels by Selma Lagerl&ouml;f from this URL:
<a href="https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt">
    <tt>https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt</tt>
</a>. The text of these novels was extracted
from <a href="https://litteraturbanken.se/forfattare/LagerlofS/titlar">Lagerlöf arkivet</a> at
<a href="https://litteraturbanken.se/">Litteraturbanken</a>.

In [2]:
! curl GET https://raw.githubusercontent.com/pnugues/ilppp/master/programs/corpus/Selma.txt > corpus/Selma.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: GET
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5294k  100 5294k    0     0  2234k      0  0:00:02  0:00:02 --:--:-- 2234k


In [5]:
# You may have to adjust the path
corpus = open('./corpus/Selma.txt', encoding='utf8').read()

Run the <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch02/python">concordance
program </a> to print the lines containing a specific word, for instance <i>Nils</i>.

In [7]:
pattern = 'Nils Holgersson'
width = 25

In [8]:
# spaces match tabs and newlines
pattern = re.sub(' ', r'\\s+', pattern)
# Replaces newlines with spaces in the text
clean_corpus = re.sub(r'\s+', ' ', corpus)
concordance = ('(.{{0,{width}}}{pattern}.{{0,{width}}})'
               .format(pattern=pattern, width=width))
for match in re.finditer(concordance, clean_corpus):
    print(match.group(1))
# print the string with 0..width characters on either side

Selma Lagerlöf Nils Holgerssons underbara resa genom Sv
! Se på Tummetott! Se på Nils Holgersson Tummetott!» Genast vände
r,» sade han. »Jag heter Nils Holgersson och är son till en husma
lden. »Inte är det värt, Nils Holgersson, att du är ängslig eller
 i dem. På den tiden, då Nils Holgersson drog omkring med vildgäs
ulle allt visa honom vad Nils Holgersson från Västra Vemmenhög va
om ägde rum det året, då Nils Holgersson for omkring med vildgäss
m vad det kan kosta dem. Nils Holgersson hade inte haft förstånd 
de det inte mer sägas om Nils Holgersson, att han inte tyckte om 
 Rosenbom?» För där stod Nils Holgersson mitt uppe på Rosenboms n
 Med ens fingo de syn på Nils Holgersson, och då sköt den store v
vila. När vildgässen och Nils Holgersson äntligen hade letat sig 
 slags arbetare. Men vad Nils Holgersson inte såg, det var, att s
nde han fråga, och om då Nils Holgersson sade nej, började han ge
de lille Mats, och om nu Nils Holgersson också hade tegat, så had
åg så försmädlig ut,

Run a simple <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">tokenization
program</a> on your corpus.

In [9]:
def tokenize(text):
    words = re.findall(r'\p{L}+', text)
    return words

In [10]:
words = tokenize(corpus)
words[:10]

['Selma',
 'Lagerlöf',
 'Nils',
 'Holgerssons',
 'underbara',
 'resa',
 'genom',
 'Sverige',
 'Första',
 'bandet']

Count the number of unique words in the original corpus and when setting all the words in lowercase

Original text

In [11]:
# Write your code here
len(list(set(words)))

44256

Lowercased text

In [12]:
# Write your code here
len(set([word.lower() for word in words]))

41032

### Segmenting a corpus

You will write a program to tokenize your text, insert `<s>` and `</s>` tags to delimit sentences, and set all the words in lowercase letters. In the end, you will only keep the words.

#### Normalizing 

Write a regular expression that matches all the characters that are neither a letter nor a punctuation sign. The punctuations signs will be the followings: `.;:?!`. In your regex, use the same order. For the definition of a letter, use a Unicode regex. You will call the regex string `nonletter`

In [88]:
# Write your code
def clean(text):
    nonletter = r'[^(\p{L}|\p{Z}|,\.!?)]'
    return re.sub(nonletter, ' ', text)


Write a `clean()` function that replaces all the characters that are neither a letter nor a punctuation sign with a space. The punctuations signs will be the followings: `.;:?!`.   For the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

the result will be:

`En gång hade de på Mårbacka en barnpiga som hette Back Kajsa.`

In [89]:
# Write your code here
clean('En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa.')

'En gång hade de på Mårbacka en barnpiga, som hette Back Kajsa.'

In [90]:
test_para = 'En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa. \
Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, \
hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, \
när hon kammade dem, och till humöret var hon dyster och sorgbunden.'

In [91]:
test_para = clean(test_para)
test_para

'En gång hade de på Mårbacka en barnpiga, som hette Back Kajsa. Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, när hon kammade dem, och till humöret var hon dyster och sorgbunden.'

#### Segmenter

In this section, you will write a sentence segmenter that will delimit each sentence with `</s>` and `<s>` symbols. For example the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

will be bracketed as:

`<s> En gång hade de på Mårbacka en barnpiga som hette Back-Kajsa </s>`

As algorithm, you will use a simple heuristics to detect the sentence boundaries: A sentence starts with a capital letter and ends with a period-equivalent punctuation sign. You will write a regex to match these boundaries with a regular expression and you will insert `</s>\n<s>` symbols with a substitution function.

In [127]:
print(test_para)

En gång hade de på Mårbacka en barnpiga, som hette Back Kajsa. Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, när hon kammade dem, och till humöret var hon dyster och sorgbunden.


##### Detecting sentence boundaries

Write a regular expression that matches a punctuation, a sequence of spaces, and an uppercase letter. Call this regex string `sentence_boundaries`. In the regex, you will remember the value of the uppercase letter using a backreference. Use the Unicode regexes for the letters and the spaces.

In [153]:
# Write your code here
sentence_boundaries = r'\p{P}\p{z}*(\p{Lu})'

##### Replacement markup

Write a string to replace the matched boundaries with the sentence boundary markup. Remember that a sentence ends with `</s>` and starts with `<s>` and that there is one sentence per line. Hint: The markup is `</s>\n<s>`. Remember also that the first letter of your sentence is in a regex backreference. Call the regex string `sentence_markup`.

In [158]:
# Write your code here
sentence_markup = r' </s>\n<s> \1'


##### Applying the substitution

Use your regexes to segment your text. Use the string `sentence_boundaries`, `sentence_markup`, and `test_para` as input and `text` as output.

In [159]:
# Write your code here
def segment(text):
    match = re.sub(sentence_boundaries, sentence_markup, text)
    return f'<s> {match} </s>'

In [160]:
test_para

'En gång hade de på Mårbacka en barnpiga, som hette Back Kajsa. Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, när hon kammade dem, och till humöret var hon dyster och sorgbunden.'

Insert markup codes in the beginning and end of the text

In [194]:
# Write your code here
text = segment(test_para)
print(text)

<s> En gång hade de på Mårbacka en barnpiga, som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, när hon kammade dem, och till humöret var hon dyster och sorgbunden. </s>


Replace the space duplicates with one space and remove the punctuation signs. For the spaces, use the Unicode regex.

In [212]:
# Write your code here
punctuation = r'\p{Whitespace}{2,}|[^[/|\P{P}]]'
text = re.sub(punctuation, '', text)

In [213]:
print(text)

<s> En gång hade de på Mårbacka en barnpiga, som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, när hon kammade dem, och till humöret var hon dyster och sorgbunden. </s>


Write a `segment_sentences(text)` function to gather the code in the Segmenter section and set the text in lowercase

In [416]:
# Write your code here
def segment_sentences(text):
    punctuation = r'[^(/|\P{P})]'
    whitespace = r'\p{Whitespace}{2,}'
    sentence_markup = r' </s>\n<s> \1'
    sentence_boundaries = r'\p{P}\p{Whitespace}*(\p{Lu})'
    match = re.sub(sentence_boundaries, sentence_markup, text)
    match = re.sub(punctuation, '', match)
    match = re.sub(whitespace, ' ', match)
    match = f'<s> {match}</s>'
    return match.lower()

In [417]:
print(segment_sentences(test_para))

<s> en gång hade de på mårbacka en barnpiga som hette back kajsa </s>
<s> hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden</s>


Estimate roughly the accuracy of your program.

#### Tokenizing the corpus

Clean and segment the corpus

In [418]:
# Write your code here
segmented_corpus = segment_sentences(clean(corpus))

The result should be a normalized text without punctuation signs where all the sentences are delimited with `<s>` and `</s>` tags. The five last lines of the text should look like this.

In [419]:
print(segmented_corpus[-557:])

<s> hon hade fått större kärlek av sina föräldrar än någon annan han visste och sådan kärlek måste vändas i välsignelse </s>
<s> då prästen sade detta kom alla människor att se bort mot klara gulla och de förundrade sig över vad de såg </s>
<s> prästens ord tycktes redan ha gått i uppfyllelse </s>
<s> där stod klara fina gulleborg ifrån skrolycka hon som var uppkallad efter själva solen vid sina föräldrars grav och lyste som en förklarad </s>
<s> hon var likaså vacker som den söndagen då hon gick till kyrkan i den röda klänningen om inte vackrare </s>


You will now create a list of words from your string. You will consider that a space or a carriage return is an item separator

In [420]:
# Write your code here
def tokenize(text):
    return re.findall(r'[^\p{Whitespace}]+', text)
words = tokenize(segmented_corpus)

The five last lines of the corpus should like this:

In [421]:
print(words[-101:])

['<s>', 'hon', 'hade', 'fått', 'större', 'kärlek', 'av', 'sina', 'föräldrar', 'än', 'någon', 'annan', 'han', 'visste', 'och', 'sådan', 'kärlek', 'måste', 'vändas', 'i', 'välsignelse', '</s>', '<s>', 'då', 'prästen', 'sade', 'detta', 'kom', 'alla', 'människor', 'att', 'se', 'bort', 'mot', 'klara', 'gulla', 'och', 'de', 'förundrade', 'sig', 'över', 'vad', 'de', 'såg', '</s>', '<s>', 'prästens', 'ord', 'tycktes', 'redan', 'ha', 'gått', 'i', 'uppfyllelse', '</s>', '<s>', 'där', 'stod', 'klara', 'fina', 'gulleborg', 'ifrån', 'skrolycka', 'hon', 'som', 'var', 'uppkallad', 'efter', 'själva', 'solen', 'vid', 'sina', 'föräldrars', 'grav', 'och', 'lyste', 'som', 'en', 'förklarad', '</s>', '<s>', 'hon', 'var', 'likaså', 'vacker', 'som', 'den', 'söndagen', 'då', 'hon', 'gick', 'till', 'kyrkan', 'i', 'den', 'röda', 'klänningen', 'om', 'inte', 'vackrare', '</s>']


### Counting unigrams and bigrams

Read and try programs to compute the frequency of unigrams and bigrams of the training set: [<a
            href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>].

#### Unigrams

In [422]:
def unigrams(words):
    frequency = {}
    for i in range(len(words)):
        if words[i] in frequency:
            frequency[words[i]] += 1
        else:
            frequency[words[i]] = 1
    return frequency

We compute the frequencies.

In [529]:
frequency = unigrams(words)
list(frequency.items())[:20]

[('<s>', 59326),
 ('selma', 52),
 ('lagerlöf', 270),
 ('nils', 87),
 ('holgerssons', 6),
 ('underbara', 23),
 ('resa', 317),
 ('genom', 688),
 ('sverige', 56),
 ('</s>', 59326),
 ('första', 525),
 ('bandet', 6),
 ('bokutgåva', 11),
 ('albert', 15),
 ('bonniers', 11),
 ('förlag', 11),
 ('stockholm', 77),
 ('den', 11623),
 ('kristliga', 2),
 ('dagvisan', 2)]

#### Bigrams

In [424]:
def bigrams(words):
    bigrams = []
    for i in range(len(words) - 1):
        bigrams.append((words[i], words[i + 1]))
    frequency_bigrams = {}
    for i in range(len(words) - 1):
        if bigrams[i] in frequency_bigrams:
            frequency_bigrams[bigrams[i]] += 1
        else:
            frequency_bigrams[bigrams[i]] = 1
    return frequency_bigrams

In [528]:
frequency_bigrams = bigrams(words)
list(frequency_bigrams.items())[:20]

[(('<s>', 'selma'), 12),
 (('selma', 'lagerlöf'), 11),
 (('lagerlöf', 'nils'), 1),
 (('nils', 'holgerssons'), 6),
 (('holgerssons', 'underbara'), 4),
 (('underbara', 'resa'), 4),
 (('resa', 'genom'), 6),
 (('genom', 'sverige'), 5),
 (('sverige', '</s>'), 17),
 (('</s>', '<s>'), 59325),
 (('<s>', 'första'), 11),
 (('första', 'bandet'), 1),
 (('bandet', 'bokutgåva'), 2),
 (('bokutgåva', 'albert'), 11),
 (('albert', 'bonniers'), 11),
 (('bonniers', 'förlag'), 11),
 (('förlag', '</s>'), 11),
 (('<s>', 'stockholm'), 13),
 (('stockholm', '</s>'), 26),
 (('<s>', 'den'), 1359)]

In the report, tell what is the possible number of bigrams and their real number? Explain why such a difference. What would be the possible number of 4-grams.

Propose a solution to cope with bigrams unseen in the corpus.

### Computing the likelihood of a sentence

#### Unigrams

Write a program to compute a sentence's probability using unigrams. You may find useful the dictionaries that we saw in the mutual information program: [<a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>]. Your function will return the perplexity.

Your function should print and tabulate the results as in the examples below with the sentence _Det var en gång en katt som hette Nils_.

```
=====================================================
wi 	 C(wi) 	 #words 	 P(wi)
=====================================================
det 	 21108 	 1041631 	 0.0202643738521607
var 	 12090 	 1041631 	 0.01160679741674355
en 	 13514 	 1041631 	 0.01297388422579589
gång 	 1332 	 1041631 	 0.001278763784871994
en 	 13514 	 1041631 	 0.01297388422579589
katt 	 16 	 1041631 	 1.5360525944408337e-05
som 	 16288 	 1041631 	 0.015637015411407686
hette 	 97 	 1041631 	 9.312318853797554e-05
nils 	 87 	 1041631 	 8.352285982272032e-05
</s> 	 59047 	 1041631 	 0.056687060964967444
=====================================================
Prob. unigrams:	 5.361459667285409e-27
Geometric mean prob.: 0.0023600885848765307
Entropy rate:	 8.726943273141258
Perplexity:	 423.71290908655254
```

In [473]:
from functools import reduce
import numpy as np
def unigram_lm(frequency, words):
    N = sum(list(frequency.values()))
    n = len(words)
    delimiter = '====================================================='
    print(delimiter + '\n' + 'wi      C(wi)      #words      P(wi) \n' + delimiter)
    ps = []
    for word in words:
        p = frequency[word] / N
        ps.append(p)
        print(word + '      ' + str(frequency[word]) + '      ' + str(N) + '      ' + str(p))
    prob_uni = reduce((lambda x, y: x * y), ps)
    entropy_rate = -1/n * np.log2(prob_uni)
    perplexity = np.power(2,entropy_rate)
    print(delimiter + f'\nProb. unigrams: {prob_uni}'\
          + f'\nGeometric mean prob.: {float(np.power(prob_uni, 1/n))}'\
          + f'\nEntropy rate: {entropy_rate}'\
          + f'\nPerprelixty: {perplexity}')
    return perplexity

In [474]:
sentence = 'det var en gång en katt som hette nils </s>'
sent_words = sentence.split()
sent_words

['det', 'var', 'en', 'gång', 'en', 'katt', 'som', 'hette', 'nils', '</s>']

In [475]:
frequency = unigrams(words)
perplexity_unigrams = unigram_lm(frequency, sent_words)

wi      C(wi)      #words      P(wi) 
det      21108      1042133      0.020254612415114
var      12090      1042133      0.011601206371931414
en      13514      1042133      0.012967634649320192
gång      1332      1042133      0.0012781477987934362
en      13514      1042133      0.012967634649320192
katt      16      1042133      1.5353126712233466e-05
som      16288      1042133      0.01562948299305367
hette      97      1042133      9.30783306929154e-05
nils      87      1042133      8.348262649776947e-05
</s>      59326      1042133      0.05692747470812267
Prob. unigrams: 5.360900589783176e-27
Geometric mean prob.: 0.0023600639734006846
Entropy rate: 8.726958317932167
Perprelixty: 423.71732769560083


In [476]:
perplexity_unigrams = int(perplexity_unigrams)
perplexity_unigrams

423

#### Bigrams

Write a program to compute the sentence probability using bigrams. Your function will tabulate and print the results as below. It will return the perplexity.

```
=====================================================
wi 	 wi+1 	 Ci,i+1 	 C(i) 	 P(wi+1|wi)
=====================================================
<s>	 det 	 5672 	 59047 	 0.09605907158704083
det 	 var 	 3839 	 21108 	 0.1818741709304529
var 	 en 	 712 	 12090 	 0.058891645988420185
en 	 gång 	 706 	 13514 	 0.052242119283705785
gång 	 en 	 20 	 1332 	 0.015015015015015015
en 	 katt 	 6 	 13514 	 0.0004439840165754033
katt 	 som 	 2 	 16 	 0.125
som 	 hette 	 45 	 16288 	 0.002762770137524558
hette 	 nils 	 0 	 97 	 0.0 	 *backoff: 	 8.352285982272032e-05
nils 	 </s> 	 2 	 87 	 0.022988505747126436
=====================================================
Prob. bigrams:	 2.376007803503683e-19
Geometric mean prob.: 0.013727289294133601
Entropy rate:	 6.186809422848149
Perplexity:	 72.84759420254609
```

In [518]:
# Write your code
from functools import reduce
import numpy as np
def bigram_lm(frequency, frequency_bigrams, words):
    N = sum(list(frequency.values()))
    n = len(words)
    delimiter = '====================================================='
    print(delimiter + '\n' + 'wi 	 wi+1 	 Ci,i+1 	 C(i) 	 P(wi+1|wi)\n' + delimiter)
    ps = []
    for bigram_ in bigrams(words):
        p = frequency_bigrams[bigram_] / frequency[bigram_[0]] if bigram_ in frequency_bigrams \
        else frequency[bigram_[1]] / N
        ps.append(p)
        print(bigram_[0] + '      '\
              + bigram_[1] + '      '\
              + str(frequency_bigrams[bigram_] if bigram_ in frequency_bigrams else 0) + '      '\
              + str(frequency[bigram_[0]]) + '      '\
              + str(N) + '      '\
              + str(p if bigram_ in frequency_bigrams else f'0.0\t*backoff {frequency[bigram_[1]] / N}'))
    prob_uni = frequency[words[0]] / N * reduce((lambda x, y: x * y), ps[1:])
    entropy_rate = -1/n * np.log2(prob_uni)
    perplexity = np.power(2,entropy_rate)
    print(delimiter + f'\nProb. unigrams: {prob_uni}'\
          + f'\nGeometric mean prob.: {float(np.power(prob_uni, 1/n))}'\
          + f'\nEntropy rate: {entropy_rate}'\
          + f'\nPerprelixty: {perplexity}')
    return np.round(perplexity)

In [519]:
perplexity_bigrams = bigram_lm(frequency, frequency_bigrams, sent_words)

wi 	 wi+1 	 Ci,i+1 	 C(i) 	 P(wi+1|wi)
det      var      3839      21108      1042133      0.1818741709304529
var      en      712      12090      1042133      0.058891645988420185
en      gång      706      13514      1042133      0.052242119283705785
gång      en      20      1332      1042133      0.015015015015015015
en      katt      6      13514      1042133      0.0004439840165754033
katt      som      2      16      1042133      0.125
som      hette      45      16288      1042133      0.002762770137524558
hette      nils      0      97      1042133      0.0	*backoff 8.348262649776947e-05
nils      </s>      2      87      1042133      0.022988505747126436
Prob. unigrams: 2.7532974741265584e-19
Geometric mean prob.: 0.0139310969190684
Entropy rate: 6.165547331161066
Perprelixty: 71.78185650486967


In [520]:
perplexity_bigrams = int(perplexity_bigrams)
perplexity_bigrams

72

In [531]:
test_sentences = ["Hej jag heter Nils",
                  "Idag är det tisdag",
                  "hundar kan vara farliga"]
test_words = [sentence.lower().split() for sentence in test_sentences]
test_perplexity_bigrams = [bigram_lm(frequency, frequency_bigrams, sent_words) for sent_words in test_words]

wi 	 wi+1 	 Ci,i+1 	 C(i) 	 P(wi+1|wi)
hej      jag      0      3      1042133      0.0	*backoff 0.009126474260003282
jag      heter      4      9511      1042133      0.00042056566081379455
heter      nils      1      78      1042133      0.01282051282051282
Prob. unigrams: 1.552162952232786e-11
Geometric mean prob.: 0.0019848803317696223
Entropy rate: 8.976732254607093
Perprelixty: 503.80871027546954
wi 	 wi+1 	 Ci,i+1 	 C(i) 	 P(wi+1|wi)
idag      är      0      1      1042133      0.0	*backoff 0.006035697938746782
är      det      688      6290      1042133      0.10937996820349762
det      tisdag      0      21108      1042133      0.0	*backoff 5.7574225170875506e-06
Prob. unigrams: 6.042862972894413e-13
Geometric mean prob.: 0.0008816793852883758
Entropy rate: 10.147458251368949
Perprelixty: 1134.1991393764124
wi 	 wi+1 	 Ci,i+1 	 C(i) 	 P(wi+1|wi)
hundar      kan      0      24      1042133      0.0	*backoff 0.0019939873317513217
kan      vara      86      2078      1042133     

In addition to this sentence, _Det var en gång en katt som hette Nils_, write three other sentences that will form your test set and run your programs on them. You will insert them in your report.

### Online prediction of words

You will now carry out an online prediction of words. You will consider two cases:
1. Prediction of the current word a user is typing;
2. Prediction of the next word.

Ideally, you would write a loop that reads the words and apply the models while typing. As the Jupyter labs are not designed for interactive input and output, we will simplify the experimental settings with constant strings at a given time of the input.  

We will assume the user is typing the phrase: _Det var en gång_. 

#### Trigrams

To have a more accurate prediction, you will use a trigram counting function. Program this function following the model of your bigram counting function.

In [541]:
# Write your code
def trigrams(words):
    trigrams = [tuple(words[idx:idx + 3])
                for idx in range(len(words) - 2)]
    frequencies = {}
    for trigram in trigrams:
        if trigram in frequencies:
            frequencies[trigram] += 1
        else:
            frequencies[trigram] = 1
    return frequencies

In [542]:
frequency_trigrams = trigrams(words)
frequency_trigrams[('det', 'var', 'en')]

330

#### Prediction

The user starts typing _Det var en gång_. After the 2nd character, your program tries to help the user with suggested words.

In [543]:
starting_text = 'De'.lower()
starting_text

'de'

Write a program to rank the five first candidates at this point. Assign these predictions in a list that you will call `current_word_predictions_1`. Note that you are starting a sentence and you can then use the bigram frequencies. Write a sorting key that will enable you have a deterministic ranking of the words or bigrams with identical frequencies: When two words have the same frequency, you will sort them by alphabetic order. You can do this with a tuple.

In [706]:
cand_nbr = 5

In [707]:
# Write your code here
def get_canditates(cand_nbr, starting_text, frequency_unigrams, frequency_bigrams):
    candidates = []
    for token in filter(lambda x: x.startswith(starting_text), list(frequency_unigrams.keys())):
        p = frequency_bigrams[('<s>',token)] if ('<s>',token) in frequency_bigrams else 0
        candidates.append((token, p))
        
    return [pair[0] for pair in sorted(candidates, key = lambda e: (-e[1],e[0]) , reverse = False)[:cand_nbr]] 
#           

In [708]:
current_word_predictions_1 = get_canditates(cand_nbr, starting_text, frequency, frequency_bigrams)

In [709]:
current_word_predictions_1

['det', 'de', 'den', 'detta', 'denna']

Let us now suppose that the user has typed: _Det var en_. After detecting a space, your program starts predicting a next possible word.

In [710]:
current_text = "Det var en ".lower()
current_text

'det var en '

Tokenize this text and return a list of tokens. Call it `tokens`.

In [711]:
# Write your code here
tokens = tokenize(current_text)

In [712]:
tokens

['det', 'var', 'en']

Write a program to propose the five next possible words ranked by frequency using a trigram model. Assign these predictions to a variable that you will call `next_word_predictions`. Write a sorting key that will enable you have a deterministic ranking of the words or bigrams with identical frequencies: When two words have the same frequency, you will sort them by alphabetic order. You can do this with a tuple.

In [747]:
# Write your code here
def get_next_word_predictions(cand_nbr, starting_text, frequency_trigrams, frequency_bigrams, frequency_unigrams):
    starting_text = [starting_text] if isinstance(starting_text, str) else starting_text
    text = tuple(starting_text[-2:])
    candidates = []
    for token in frequency_unigrams:
        gram = (*text, token) 
        if tuple(text) in frequency_bigrams and gram in frequency_trigrams:
            p = frequency_trigrams[gram] / frequency_bigrams[text]
        elif gram in frequency_bigrams and text[0] in frequency:
            p = frequency_bigrams[gram] / frequency[text[0]]
        else:
            p = 0
        candidates.append((token, p))
    return [pair[0] for pair in sorted(candidates, key = lambda e: (-e[1],e[0]) , reverse = False)[:cand_nbr]]

In [748]:
next_word_predictions = get_next_word_predictions(cand_nbr, tokens, frequency_trigrams, frequency_bigrams, frequency)
next_word_predictions

['stor', 'liten', 'gammal', 'god', 'sådan']

Finally, let us suppose that the user has typed _Det var en g_, rank the five possible candidates. Assign these predictions in a list that you will call `current_word_predictions_2`

In [749]:
current_text = "Det var en g".lower()

In [750]:
# Write your code here
def current_word_predictions_2(cand_nbr, starting_text, frequency, frequency_bigrams, frequency_trigrams):
    starting_text = tokenize(starting_text)
    filtered_unigrams = {k:v for k,v in frequency.items() if k.startswith(starting_text[-1])}
    return get_next_word_predictions(cand_nbr, tokens, frequency_trigrams, frequency_bigrams, filtered_unigrams)
    

In [751]:
current_word_predictions_2 = current_word_predictions_2(cand_nbr, current_text, frequency, frequency_bigrams, frequency_trigrams)
current_word_predictions_2

['gammal', 'god', 'gång', 'ganska', 'glädje']

## Checked answers

The automatic marking system will check these answers: `(perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2)`

In [752]:
(perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2)

(423,
 72,
 ['det', 'de', 'den', 'detta', 'denna'],
 ['stor', 'liten', 'gammal', 'god', 'sådan'],
 ['gammal', 'god', 'gång', 'ganska', 'glädje'])

## Submission

When you have written all the code and run all the cells, fill in your ID and as well as the name of the notebook.

In [753]:
STIL_ID = ["ni5324ro-s", "si7660da-s"] # Write your stil ids as a list
CURRENT_NOTEBOOK_PATH = os.path.join(os.getcwd(), 
                                     "lab_2.ipynb") # Write the name of your notebook

The submission code will send your answer. It consists of the perplexities and predictions.

In [754]:
import json
ANSWER = json.dumps({'perplexity_unigrams': perplexity_unigrams,
                     'perplexity_bigrams': perplexity_bigrams,
                     'current_word_predictions_1': current_word_predictions_1,
                     'next_word_predictions': next_word_predictions,
                     'current_word_predictions_2': current_word_predictions_2})
ANSWER

'{"perplexity_unigrams": 423, "perplexity_bigrams": 72, "current_word_predictions_1": ["det", "de", "den", "detta", "denna"], "next_word_predictions": ["stor", "liten", "gammal", "god", "s\\u00e5dan"], "current_word_predictions_2": ["gammal", "god", "g\\u00e5ng", "ganska", "gl\\u00e4dje"]}'

The checking script will accept perplexities with a margin, like for instance:
```
(421, 72, ['det', 'de', 'den', 'detta', 'denna'], ['stor', 'liten', 'gammal', 'god', 'sådan'], ['gammal', 'god', 'gång', 'ganska', 'glädje'])
```

Now the moment of truth:
1. Save your notebook and
2. Run the cells below

In [755]:
SUBMISSION_NOTEBOOK_PATH = CURRENT_NOTEBOOK_PATH + ".submission.bz2"

In [756]:
ASSIGNMENT = 2
API_KEY = "f581ba347babfea0b8f2c74a3a6776a7"

# Copy and compress current notebook
with bz2.open(SUBMISSION_NOTEBOOK_PATH, mode="wb") as fout:
    with open(CURRENT_NOTEBOOK_PATH, "rb") as fin:
        fout.write(fin.read())

In [757]:
res = requests.post("https://vilde.cs.lth.se/edan20checker/submit", 
                    files={"notebook_file": open(SUBMISSION_NOTEBOOK_PATH, "rb")}, 
                    data={
                        "stil_id": STIL_ID,
                        "assignment": ASSIGNMENT,
                        "answer": ANSWER,
                        "api_key": API_KEY,
                    },
                   verify=True)

# from IPython.display import display, JSON
res.json()

{'msg': None,
 'status': 'correct',
 'signature': '736dd7fbdf9104ed569f4760798c84dfa6f2d5068eb93a64285b4493f58d72b68cc56e9ccbac9bf5a1ae85f3308702ed7f5dbcec630eff17ffbddb79d5e29eab',
 'submission_id': '239df2e2-8ca3-4550-b130-608634909c27'}

## Turning in your assignment

Now your are done with the program. To complete this assignment, you will:
1. Write a short individual report on your program. You will include the regular expression you use to segment the text and the unigram and bigram tables for three sentences and _Det var en gång en katt som hette Nils_.
2. Execute the Jupyter notebook by Peter Norvig here: <a href="http://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb">https://nbviewer.jupyter.org/url/norvig.com/ipython/How to Do Things with Words.ipynb</a>. Just run all the cells and be sure that you understand the code. You will find the data here: <a href="http://norvig.com/ngrams/">http://norvig.com/ngrams/</a>.
3. In your report, after the description of your program, you will describe one experiment with Norvig's notebook and a long string of words your will create yourself or copy from a text you like. You will remove all the punctuation and white spaces from this string. You will set this string in lowercase letters. You will just add a cell at the end of Sect. 7 in Norvig's notebook, where you will use your string and run the notebook cell with the <tt>segment()</tt> and <tt>segment2()</tt> functions. You will comment the segmentation results you obtain with the unigram and bigram models.

Submit your report as well as your notebook (for archiving purposes) to Canvas: https://canvas.education.lu.se/. To write your report, you can either
1. Write directly your text in Canvas, or
2. Use Latex and Overleaf (www.overleaf.com). This will probably help you structure your text. You will then upload a PDF file in Canvas.

The submission deadline is September 23, 2022. You will have only three submission attempts. The deadline for the second and third ones are one week after you are noticed of your result.